In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import concatenate, Dense, Input, Flatten
from keras.optimizers import Adam
import gym
from rl.agents import eventDDPGAgent
from rl.memory import SequentialMemory

/Users/admin/.pyenv/versions/3.6.6/lib/python3.6/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
Using TensorFlow backend.


In [2]:
# GymのPendulum環境を作成
env = gym.make("Pendulum-v0")

# 取りうる”打ち手”のアクション数と値の定義
nb_actions = 2
ACT_ID_TO_VALUE = {0: [-1], 1: [+1]}

print("Action Space: %s" % env.action_space)
#action  dim = 1
#critic dim = 3 with ??
print( env.observation_space.shape[0])

Action Space: Box(1,)
3


In [3]:
def actor_net(a_shape, s_shape):
    action_input = Input(shape=(1,)+s_shape)
    x = Flatten()(action_input)
    x = Dense(32, activation="relu")(x)
    x = Dense(16, activation="relu")(x)
    x = Dense(3, activation="tanh")(x)
    #x = Dense(a_shape[0], activation="linear")(x)
    actor = Model(inputs=action_input, outputs=x)
    return actor

In [4]:
def critic_net(a_shape , s_shape):
    action_input = Input(a_shape)
    observation_input = Input(shape=(1,)+s_shape)
    flattened_observation = Flatten()(observation_input)
    x = concatenate([action_input, flattened_observation])
    x = Dense(32, activation="relu")(x)
    x = Dense(32, activation="relu")(x)
    x = Dense(1, activation="linear")(x)
    critic = Model(inputs=[action_input, observation_input], outputs=x)
    return (critic, action_input)


In [5]:
def agent(a_shape, s_shape):
    actor = actor_net(a_shape, s_shape)
    critic,  critic_action_input = critic_net(a_shape, s_shape)
    memory = SequentialMemory(limit = 50000, window_length = 1)
    print('critic_action_input = ', critic_action_input)
    agent = eventDDPGAgent(
        a_shape[0],
        actor,
        critic,
        critic_action_input,
        memory
    )
    return agent

In [6]:
agent = agent((3,), env.observation_space.shape)
print(env.action_space.shape, env.observation_space.shape)
agent.compile(Adam(lr=0.001, clipnorm=1.), metrics=["mae"])
agent.fit(env, nb_steps=100000, visualize=True, verbose=1, nb_max_episode_steps=200)
print('Finish Learning. We start test phase.')
agent.test(env, nb_episodes=5, visualize=True, nb_max_episode_steps=200)

W0924 16:54:45.896171 140736146887552 deprecation_wrapper.py:119] From /Users/admin/.pyenv/versions/3.6.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0924 16:54:45.941265 140736146887552 deprecation_wrapper.py:119] From /Users/admin/.pyenv/versions/3.6.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0924 16:54:45.992090 140736146887552 deprecation_wrapper.py:119] From /Users/admin/.pyenv/versions/3.6.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4158: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0924 16:54:46.149750 140736146887552 deprecation_wrapper.py:119] From /Users/admin/.pyenv/versions/3.6.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is de

network.py shape =  (None, 1, 3)
network.py shape =  (None, 3)
network.py shape =  (None, 1, 3)
critic_action_input =  Tensor("input_2:0", shape=(?, 3), dtype=float32)
(1,) (3,)


W0924 16:54:46.559592 140736146887552 deprecation_wrapper.py:119] From /Users/admin/.pyenv/versions/3.6.6/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



network.py shape =  (None, 1, 3)
network.py shape =  (None, 3)
network.py shape =  (None, 1, 3)
Training for 100000 steps ...

episode =  0
Interval 1 (0 steps performed)
  199/10000 [..............................] - ETA: 4:03 - reward: -5.9631
episode =  1
  399/10000 [>.............................] - ETA: 3:19 - reward: -6.9572
episode =  2
  599/10000 [>.............................] - ETA: 3:02 - reward: -6.7801
episode =  3
  797/10000 [=>............................] - ETA: 2:52 - reward: -6.6200
episode =  4
  999/10000 [=>............................] - ETA: 2:44 - reward: -6.5039
episode =  5
 1200/10000 [==>...........................] - ETA: 2:49 - reward: -6.4109
episode =  6
 1399/10000 [===>..........................] - ETA: 2:42 - reward: -6.3138
episode =  7
 1599/10000 [===>..........................] - ETA: 2:37 - reward: -6.3208
episode =  8
 1797/10000 [====>.........................] - ETA: 2:31 - reward: -6.1485
episode =  9
 1998/10000 [====>...................

 7798/10000 [======================>.......] - ETA: 36s - reward: -5.7294
episode =  89
 7999/10000 [======================>.......] - ETA: 33s - reward: -5.6814
episode =  90
 8198/10000 [=======================>......] - ETA: 30s - reward: -5.6211
episode =  91
 8400/10000 [========================>.....] - ETA: 26s - reward: -5.5755
episode =  92
 8598/10000 [========================>.....] - ETA: 23s - reward: -5.5388
episode =  93
 8799/10000 [=========================>....] - ETA: 20s - reward: -5.5130
episode =  94
 8997/10000 [=========================>....] - ETA: 16s - reward: -5.4873
episode =  95
 9198/10000 [==========================>...] - ETA: 13s - reward: -5.4515
episode =  96
 9400/10000 [===========================>..] - ETA: 10s - reward: -5.4169
episode =  97
 9600/10000 [===========================>..] - ETA: 6s - reward: -5.3806
episode =  98
 9798/10000 [============================>.] - ETA: 3s - reward: -5.3661
episode =  99
10000/10000 [=====================

 5400/10000 [===============>..............] - ETA: 1:16 - reward: -4.3787
episode =  177
 5598/10000 [===============>..............] - ETA: 1:13 - reward: -4.4397
episode =  178
 5799/10000 [================>.............] - ETA: 1:10 - reward: -4.4979
episode =  179
 6000/10000 [=================>............] - ETA: 1:06 - reward: -4.5686
episode =  180
 6199/10000 [=================>............] - ETA: 1:03 - reward: -4.4616
episode =  181
 6398/10000 [==================>...........] - ETA: 1:00 - reward: -4.3798
episode =  182
 6599/10000 [==================>...........] - ETA: 56s - reward: -4.4502
episode =  183
 6798/10000 [===================>..........] - ETA: 53s - reward: -4.4106
episode =  184
 6999/10000 [===================>..........] - ETA: 50s - reward: -4.4380
episode =  185
 7197/10000 [====================>.........] - ETA: 46s - reward: -4.4772
episode =  186
 7400/10000 [=====================>........] - ETA: 43s - reward: -4.4407
episode =  187
 7597/10000 [==

 2800/10000 [=======>......................] - ETA: 2:00 - reward: -2.3217
episode =  264
 3000/10000 [========>.....................] - ETA: 1:56 - reward: -2.3352
episode =  265
 3199/10000 [========>.....................] - ETA: 1:53 - reward: -2.2684
episode =  266
 3399/10000 [=========>....................] - ETA: 1:50 - reward: -2.2099
episode =  267
 3599/10000 [=========>....................] - ETA: 1:46 - reward: -2.2649
episode =  268
 3797/10000 [==========>...................] - ETA: 1:43 - reward: -2.2096
episode =  269
 3999/10000 [==========>...................] - ETA: 1:40 - reward: -2.1586
episode =  270
 4199/10000 [===========>..................] - ETA: 1:36 - reward: -2.2145
episode =  271
 4400/10000 [============>.................] - ETA: 1:33 - reward: -2.1407
episode =  272
 4599/10000 [============>.................] - ETA: 1:30 - reward: -2.0756
episode =  273
 4799/10000 [=============>................] - ETA: 1:26 - reward: -2.0157
episode =  274
 4999/1000

  396/10000 [>.............................] - ETA: 1:16 - reward: -2.9885
episode =  352
  599/10000 [>.............................] - ETA: 1:11 - reward: -3.3055
episode =  353
  796/10000 [=>............................] - ETA: 1:07 - reward: -2.6460
episode =  354
 1000/10000 [==>...........................] - ETA: 1:04 - reward: -2.1094
episode =  355
 1199/10000 [==>...........................] - ETA: 1:02 - reward: -2.1674
episode =  356
 1396/10000 [===>..........................] - ETA: 1:00 - reward: -2.1217
episode =  357
 1595/10000 [===>..........................] - ETA: 59s - reward: -1.9366
episode =  358
 1799/10000 [====>.........................] - ETA: 57s - reward: -1.8510
episode =  359
 1994/10000 [====>.........................] - ETA: 56s - reward: -1.8583
episode =  360
 2199/10000 [=====>........................] - ETA: 59:41 - reward: -1.7428
episode =  361
 2400/10000 [======>.......................] - ETA: 53:33 - reward: -1.6483
episode =  362
 2598/10000

 8400/10000 [========================>.....] - ETA: 18s - reward: -2.4748
episode =  442
 8600/10000 [========================>.....] - ETA: 16s - reward: -2.5736
episode =  443
 8800/10000 [=========================>....] - ETA: 14s - reward: -2.5573
episode =  444
 8997/10000 [=========================>....] - ETA: 11s - reward: -2.6415
episode =  445
 9200/10000 [==========================>...] - ETA: 9s - reward: -2.6144
episode =  446
 9399/10000 [===========================>..] - ETA: 7s - reward: -2.6546
episode =  447
 9600/10000 [===========================>..] - ETA: 4s - reward: -2.7393
episode =  448
 9799/10000 [============================>.] - ETA: 2s - reward: -2.8221
episode =  449
10000/10000 [==============================] - 123s 12ms/step - reward: -2.8999

episode =  450
50 episodes - episode_reward: -579.976 [-1372.586, -0.262] - loss: 10.272 - mean_absolute_error: 1.873 - mean_q: -47.096

Interval 10 (90000 steps performed)
  199/10000 [.........................

In [8]:
a = np.array([1,2]).shape
b = np.array([1,2,3]).shape
c = np.array([1,2,3,4]).shape
print(a + b + c)

(2, 3, 4)


In [9]:
a = np.array([ [ [ 1,2 ] ], [ [ 2,1 ] ] ])
print(a.shape)
print(a[0][1])

(2, 1, 2)


IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
def array(n):
    a = []
    for i in range(n):
        a.append([1,2])
    return a

print(array(3))
print(array(3)[0])
print(array[0](3))

In [ ]:
a = np.array([[1,2],[3,4],[5,6]])
b = np.zeros((3,1))
for i in range(a.shape[0]):
    b[i] = a[i][0]
print(b)

In [ ]:
a = np.array([[[1,2]],[[3,2]]])
print(a)
print(a.shape)

In [ ]:
a = 1
a[0]

In [ ]:
critic net のビルド時に_feed_input_shape が変わるのでは。 a_shape = (3,)にすれば よいい